In [1]:
# import section

import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [3]:
# Data loading function

def LoadData():
    year = 1972
    total_data = {}
    years = []
    for i in range(28):
        if year == 1979 or year == 1981 or year == 1984 or year == 1986 or year ==1992:
            year += 1
        years.append(str(year))
        print year,
        data = pd.io.stata.read_stata("/Users/Matthijsmeijers/Documents/MasterclassDATA/GSSdataproject/MasterclassBDProject/GSS" + str(year) + ".DTA")
        total_data[str(year)] = data
        year += 1
        if year > 1994:
            year += 1
        
    print "Data Loaded"
    return total_data

total_data = LoadData()

 1972 1973 1974 1975 1976 1977 1978 1980 1982 1983 1985 1987 1988 1989 1990 1991 1993 1994 1996 1998 2000 2002 2004 2006 2008 2010 2012 2014 Data Loaded


In [28]:
def AllAnswers(category):
    for year in total_data.keys():
        if category not in total_data[year].keys():
            continue
        else:
            cat = getattr(total_data[year], category)
            cont_subcats = []
            for subcat in cat:
                if subcat not in cont_subcats:
                    cont_subcats.append(subcat)
                    
    return cont_subcats



In [33]:
# Counting function
# Returns dictionary with frequency of certain answer in a chosen category

def Count(year, category):
      if category1 not in total_data[year].keys():
        return
    elif category2 not in total_data[year].keys():
        return
    else:
        filtered = getattr(total_data[year], category)
        counted = {}
        for answer in filtered:
            if answer not in counted:
                counted[answer] = 1
            else:
                counted[answer] = counted[answer] + 1   
        return counted

def CategoryFilter(year, category):
    incomplete = Count(year, category)
    totalcats = AllAnswers(category)
    for category in totalcats:
        if category not in incomplete.keys():
            incomplete[category] = 1e-16
    return incomplete
    
# Proportion function
# Returns list with proportions based on counting function
def Proportions(year, category):  # input should be output of Count function
    counted = CategoryFilter(year, category)
    numbervals = counted.values()
    totalval = 0
    proportions = []
    for item in numbervals:
        totalval += item
    for item in numbervals:
        proportions.append(float(item) / float(totalval))
        
    return proportions



IndentationError: unindent does not match any outer indentation level (<ipython-input-33-122d5177b934>, line 7)

In [34]:
def CrossCount(year, category1, category2, subcategory):
    if category1 not in total_data[year].keys():
        return
    elif category2 not in total_data[year].keys():
        return
    else:
        
        filtered = getattr(total_data[year], category1)[(getattr(total_data[year], category2)) == subcategory]
        counted = {}
        for answer in filtered:
            if answer not in counted:
                counted[answer] = 1
            else:
                counted[answer] = counted[answer] + 1   
        return counted

def CrossCategoryFilter(year, category1, category2, subcategory):
    incomplete = CrossCount(year, category1, category2, subcategory)
    totalcats = AllAnswers(category1)
    for category in totalcats:
        if category not in incomplete.keys():
            incomplete[category] = 1e-16
    return incomplete

def CrossProportions(year, category1, category2, subcategory):
    counted = CrossCategoryFilter(year, category1, category2, subcategory)
    numbervals = counted.values()
    totalval = 0
    proportions = []
    for item in numbervals:
        totalval += item
    for item in numbervals:
        proportions.append(float(item) / float(totalval))
        
    return proportions

# Entropy function

def Entropy(year, category1, category2, subcategory):
    param1 = CrossProportions(year, category1, category2, subcategory)
    param2 = Proportions(year, category1)
    if len(param1) != len(param2):
        params = [param1, param2]
        lengths = [len(param1), len(param2)]
        maxlength = max(lengths)    
        for param in params:
            if len(param) < maxlength:
                for i in range(maxlength - len(param)):
                    param.append(1*10**-20)
    entropy = sp.stats.entropy(param1, param2)
    return entropy



In [40]:
ListofCategories = ['wrkstat','agewed','wrkgovt','divorce','sibs','childs','degree','hompop','partyid','polview',
                    'natcrim','natdrug','natarms','spkath','spkrac','spkcom','spkhomo','cappun','gunlaw','courts','grass']
def ListofEntropies(ListofCategories, category2, subcategory):
    Entropylist = []
    for category1 in ListofCategories:
        #loop over the every year
        year = 1972
        entropies = []
        for i in range(28):
            if year == 1979 or year == 1981 or year == 1984 or year == 1986 or year ==1992:
                year += 1
            if category1 not in total_data[str(year)].keys():
                year += 1
                if year > 1994:
                    year += 1
                continue
            if category2 not in total_data[str(year)].keys():
                year+= 1
                if year > 1994:
                    year += 1
                continue
            entropies.append(Entropy(str(year), category1, category2, subcategory))
            year += 1
            if year > 1994:
                year += 1
        Entropylist.append((np.mean(entropies), category1))
    return Entropylist

In [44]:
ListofEntropies(ListofCategories, "relig", "catholic")
"""Do this for the four religions we have (catholic, protestant, none, and jewish) and keep the top three (or 
fewer or more..) categories for each religion that have the highes correlation.
Then make for each category/religion pair a plot with two subplots: 1. showing probability of a certain answer of 
the total population over time and the probability of the answer of the religion population over time. 2. the 
probability of the religion over time (we already have this)"""
""" only had time for this.. I'll work again on it later."""



SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-44-7abe2d56bcda>, line 2)

## 